In [1]:
import pickle

# load the complete association matrix of query-page (pre-constructed)
association_mat = pickle.load(open('data/all_question_article_ratings.pkl','rb'))

# load the complete text-id dictionary of queries (pre-constructed)
question_id = pickle.load(open('data/all_questions_dict.pkl','rb'))

# load the complete text-features dictionary of queries (pre-constructed)
question_features = pickle.load(open('data/all_questions_title.pkl','rb'))

# load the complete url-id dictionary of pages (pre-constructed)
page_id = pickle.load(open('data/infowave_allintents_dict.pkl','rb'))

# load the complete url-features dictionary of pages (pre-constructed)
page_features = pickle.load(open('data/infowave_allintents_title.pkl','rb'))

# question-intent features (pre-constructed)
question_intent_features = pickle.load(open('data/all_questions_intent.pkl','rb'))



def sample_by_page_id():
    # sample the data by page-id, here we limit a number of page ids,  only keep the page < 10
    sample_threhold = 10

    ratings=[]
    dict_question={}
    dict_question_embed={}
    dict_question_intent={}
    dict_q_mapping={}
#     dict_url={}
#     dict_title={}

    for x,y in association_mat:
        if (y<sample_threhold): # filter by page id
            ratings.append((x,y))

            #print(x,y)

    all_q_ids = [x for (x,y) in ratings]
    all_i_ids = [y for (x,y) in ratings]

    icount = 0

    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):

            dict_question[k]=v
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1

    
    new_ratings=[]

    for x,y in ratings:
        new_ratings.append((dict_q_mapping[x],y))

    ratings = new_ratings

    dict_question={}
    dict_question_embed={}

    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):

            dict_question[k]=dict_q_mapping[v]
            dict_question_embed[k]=question_features[k]
            #dict_question_intent[k]=question_intent_features[k]

    pickle.dump(ratings,open('data/sample_by_page_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_question,open('data/sample_by_page_questions_dict.pkl','wb'))
    pickle.dump(dict_question_embed,open('data/sample_by_page_questions_features.pkl','wb'))
    #pickle.dump(dict_question_intent,open('data/sample_by_page_questions_intent_features.pkl','wb'))

    
sample_by_page_id()

In [82]:
import pickle

# load the complete association matrix of query-page (pre-constructed)
association_mat = pickle.load(open('data/all_question_article_ratings.pkl','rb'))

# load the complete text-id dictionary of queries (pre-constructed)
question_id = pickle.load(open('data/all_questions_dict.pkl','rb'))

# load the complete text-features dictionary of queries (pre-constructed)
question_features = pickle.load(open('data/all_questions_title.pkl','rb'))

# load the bow features
question_bow_features = pickle.load(open('data/question_tfidf_bow.pickle','rb'))

# load the complete url-id dictionary of pages (pre-constructed)
page_id = pickle.load(open('data/infowave_allintents_dict.pkl','rb'))

# load the complete url-features dictionary of pages (pre-constructed)
page_features = pickle.load(open('data/infowave_allintents_title.pkl','rb'))

# load the complete bow features of queries (pre-constructed)
query_bow_features = pickle.load(open('data/infowave_allintents_title.pkl','rb'))

# question-intent features (pre-constructed)
question_intent_features = pickle.load(open('data/all_questions_intent.pkl','rb'))

In [83]:
question_bow_features

<996x300 sparse matrix of type '<class 'numpy.float64'>'
	with 7365 stored elements in Compressed Sparse Row format>

In [87]:
import numpy as np

X_all_intent_features = np.zeros((996,25))
for k,v in question_intent_features.items():
    X_all_intent_features[k,:]=v
    
X_all_page_pca_features = np.zeros((321,5))   

page_pca_features = pickle.load(open('data/infowave_allintents_title.pkl','rb'))
all_pca_features = np.concatenate([v for k,v in page_pca_features.items()], 0)
pca = PCA(n_components=5)
pca.fit_transform(all_pca_features)
X_pca = pca.transform(all_pca_features)

for i in range(321):
    X_all_page_pca_features[i,:]=X_pca[i,:]
    
    
y = np.zeros((996,321))

for k,v in association_mat:
    y[k,v]=1
    

train_size = 600    
test_size = 200

train_sample_list = random.sample(range(996),train_size)

X_train_intent_features = np.zeros((train_size,25))
X_train_bow_features = np.zeros((train_size,300))
Y_train_labels = np.zeros((train_size,321))

icount = 0

for i in sample_list:
    X_train_intent_features[icount,:]=X_all_intent_features[i,:]
    X_train_bow_features[icount,:] = question_bow_features[i,:].todense()
    Y_train_labels[icount,:]=y[i,:]
    icount+=1

    
sample_diff = np.setdiff1d(range(996),train_sample_list)
test_sample_list = random.sample(list(sample_diff), test_size)

X_test_intent_features = np.zeros((test_size,25))
X_test_bow_features = np.zeros((test_size,300))
Y_test_labels = np.zeros((test_size,321))

icount = 0

for i in test_sample_list:
    X_test_intent_features[icount,:]=X_all_intent_features[i,:]
    X_test_bow_features[icount,:]=question_bow_features[i,:].todense()
    Y_test_labels[icount,:]=y[i,:]
    icount+=1
    
    
pickle.dump(X_train_intent_features,open('data/X_train_intent_features.pkl','wb'))
pickle.dump(Y_train_labels,open('data/Y_train_labels.pkl','wb'))            
pickle.dump(X_train_bow_features,open('data/X_train_bow_features.pkl','wb'))
pickle.dump(X_all_page_pca_features,open('data/X_all_page_pca_features.pkl','wb'))
pickle.dump(X_test_intent_features,open('data/X_test_intent_features.pkl','wb'))
pickle.dump(X_test_bow_features,open('data/X_test_bow_features.pkl','wb'))
pickle.dump(Y_test_labels,open('data/Y_test_labels.pkl','wb'))

    

In [79]:
X_train_intent_features[1,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [52]:
import random 
random.sample
sample_list = random.sample(range(996),300)

def sample_by_query_id(sample_list):
    # sample the data by query-id, 
    # sample_threhold = 900
    
    ratings = []
    
    #dict_i_mapping={}  # for hold out validation setting, we should not shrink the page id
    
    dict_q_mapping={}  

    # construct the dictionaries 
    dict_url={}
    dict_title_embed={}  
    
    dict_question={}
    dict_question_embed={}
    dict_question_intents={}
    
    #filter out query-id and page-id by association matrix
    for x,y in association_mat:
        #if (x<sample_threhold): # filter by query id
        if (x in sample_list): # filter by query id
            ratings.append((x,y))
            
    all_q_ids = [x for (x,y) in ratings if x in sample_list]
    #all_i_ids = [y for (x,y) in ratings]            
    
    print(len(all_q_ids))
    
    icount = 0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):
            dict_question[k]=v
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1    

    
    icount = 0

    for k,v in page_id.items():
        #print(k,v)
        #if ((v[0] in all_i_ids) and (k in page_features)):
        if k in page_features:                
            #print('appending page id to dictionary')
            dict_url[k]=v[0]
            dict_title_embed[k]=page_features[k]
            
#             if v[0] not in dict_i_mapping:
#                 dict_i_mapping[v[0]]=icount
#                 icount+=1
                
#   print(dict_q_mapping)
   
    
    new_ratings=[]

    # remap the page id to the sampled new ids 
    for x,y in ratings:
        new_ratings.append((dict_q_mapping[x],y))
    
    
    # replace the rating association matrix with the one with new ID-mappings
    ratings = new_ratings

    # construct the dictionaries 
    dict_url={}
    dict_url_embed={}    

    dict_question={}
    dict_question_embed={}
    #dict_q_mapping={}

    for k,v in page_id.items():
        #if (v[0] in all_i_ids) and (k in page_features):
        if k in page_features:

            dict_url[k]=v[0]
            dict_url_embed[k]=page_features[k]    
            
    for q_id in all_q_ids:
        #print(dict_q_mapping[q_id])
        #print(q_id)
        dict_question_intents[dict_q_mapping[q_id]]=question_intent_features[q_id]            
    
    print(len(dict_question_intents))
    
    dict_q_mapping={}
    icount =0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):
            dict_question[k]=icount
            dict_question_embed[k]=question_features[k]
            
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1
                
    pickle.dump(dict_question,open('data/sample_by_question_questions_dict.pkl','wb'))
    pickle.dump(dict_question_embed,open('data/sample_by_question_questions_features.pkl','wb'))            
    pickle.dump(ratings,open('data/sample_by_question_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_url,open('data/sample_by_question_page_dict.pkl','wb'))
    pickle.dump(dict_title_embed,open('data/sample_by_question_page_features.pkl','wb'))
    pickle.dump(dict_question_intents,open('data/sample_by_question_query_intents.pkl','wb'))

sample_by_query_id(sample_list) 

299
299


In [56]:
ratings=pickle.load(open('data/sample_by_question_questions_article_ratings.pkl','rb'))

In [57]:
len(ratings)

299

In [38]:
# save the hold out data
def holdout_by_query_id(sample_list):
    # sample the data by query-id, 
    # sample_threhold = 900

    ratings = []
    #dict_i_mapping={}
    
    dict_q_mapping={}

    # construct the dictionaries 
    dict_url={}
    dict_title_embed={}  
    
    dict_question={}
    dict_question_embed={}
    dict_question_intents={}
    
    #filter out query-id and page-id by association matrix
    for x,y in association_mat:
        #if (x<sample_threhold): # filter by query id
        if (x not in sample_list): # filter by query id
            ratings.append((x,y))
            
    all_q_ids = [x for (x,y) in ratings]
    #all_i_ids = [y for (x,y) in ratings]            
    
    #print(all_i_ids)
    
    icount = 0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):

            dict_question[k]=v
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1    

    
    icount = 0

    for k,v in page_id.items():
        #print(k,v)
        #if ((v[0] in all_i_ids) and (k in page_features)):
        if k in page_features:
            #print('appending page id to dictionary')
            dict_url[k]=v[0]
            dict_title_embed[k]=page_features[k]
            
#             if v[0] not in dict_i_mapping:
#                 dict_i_mapping[v[0]]=icount
#                 icount+=1
                
    print(dict_q_mapping)
    
    new_ratings=[]

    # remap the page id to the sampled new ids 
    for x,y in ratings:
        #new_ratings.append((dict_q_mapping[x],dict_i_mapping[y]))
        new_ratings.append((dict_q_mapping[x],y))

    # replace the rating association matrix with the one with new ID-mappings
    ratings = new_ratings

    # construct the dictionaries 
    dict_url={}
    dict_url_embed={}    

#     dict_question={}
#     dict_question_embed={}

    for k,v in page_id.items():
        #if (v[0] in all_i_ids) and (k in page_features):
        if k in page_features:
            dict_url[k]=v[0]
            dict_url_embed[k]=page_features[k]    
            
    for q_id in all_q_ids:
        #print(dict_q_mapping[q_id])
        #print(q_id)
        dict_question_intents[dict_q_mapping[q_id]]=question_intent_features[q_id]            
    
    #print(dict_i_mapping)
    
    icount =0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):
            dict_question[k]=icount
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1
                
    pickle.dump(dict_question,open('data/heldout_by_question_questions_dict.pkl','wb'))
    pickle.dump(dict_question_embed,open('data/heldout_by_question_questions_features.pkl','wb'))            
    pickle.dump(ratings,open('data/heldout_by_question_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_url,open('data/heldout_by_question_page_dict.pkl','wb'))
    pickle.dump(dict_title_embed,open('data/heldout_by_question_page_features.pkl','wb'))
    pickle.dump(dict_question_intents,open('data/heldout_by_question_query_intents.pkl','wb'))
    
    
                                          
holdout_by_query_id(sample_list)
                                        

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 8: 6, 10: 7, 11: 8, 12: 9, 13: 10, 15: 11, 16: 12, 17: 13, 18: 14, 19: 15, 21: 16, 23: 17, 25: 18, 27: 19, 28: 20, 31: 21, 32: 22, 33: 23, 35: 24, 36: 25, 37: 26, 38: 27, 39: 28, 40: 29, 41: 30, 42: 31, 47: 32, 48: 33, 50: 34, 52: 35, 53: 36, 54: 37, 55: 38, 58: 39, 60: 40, 61: 41, 62: 42, 64: 43, 65: 44, 68: 45, 69: 46, 70: 47, 71: 48, 72: 49, 73: 50, 74: 51, 75: 52, 76: 53, 77: 54, 78: 55, 79: 56, 80: 57, 81: 58, 82: 59, 85: 60, 87: 61, 88: 62, 89: 63, 90: 64, 92: 65, 93: 66, 94: 67, 95: 68, 96: 69, 98: 70, 99: 71, 100: 72, 101: 73, 103: 74, 104: 75, 105: 76, 106: 77, 107: 78, 108: 79, 110: 80, 111: 81, 112: 82, 113: 83, 116: 84, 117: 85, 118: 86, 119: 87, 121: 88, 122: 89, 126: 90, 128: 91, 129: 92, 131: 93, 132: 94, 133: 95, 134: 96, 135: 97, 138: 98, 139: 99, 140: 100, 142: 101, 144: 102, 145: 103, 146: 104, 147: 105, 148: 106, 150: 107, 152: 108, 154: 109, 156: 110, 158: 111, 159: 112, 161: 113, 162: 114, 164: 115, 167: 116, 169: 117, 170: 118

In [39]:
dict_question=pickle.load(open('data/heldout_by_question_questions_dict.pkl','rb'))
dict_question_embed=pickle.load(open('data/heldout_by_question_questions_features.pkl','rb'))            
ratings=pickle.load(open('data/heldout_by_question_questions_article_ratings.pkl','rb'))
dict_url=pickle.load(open('data/heldout_by_question_page_dict.pkl','rb'))
dict_title_embed=pickle.load(open('data/heldout_by_question_page_features.pkl','rb'))
dict_question_intents=pickle.load(open('data/heldout_by_question_query_intents.pkl','rb'))

print(len(dict_question))
print(len(dict_question_intents))

test_sample=random.sample(range(len(dict_question)),200)

dict_sampled_intents = {}
ylabel = np.zeros((200,321))

icount = 0
for i in test_sample:
    dict_sampled_intents[icount]=dict_question_intents[i]
    icount +=1

icount =0
for x,y in ratings:
    if x in test_sample:
        ylabel[icount,y]=1
        icount+=1

pickle.dump(dict_sampled_intents,open('data/heldoutsample_by_question_query_intents.pkl','wb'))
pickle.dump(ylabel,open('data/heldoutsample_by_question_article_ratings.pkl','wb'))           

688
688


In [34]:
len(dict_sampled_intents)

200

In [12]:
ratings = pickle.load(open('./data/sample_by_question_questions_article_ratings.pkl','rb'))
question_id = pickle.load(open('data/sample_by_question_questions_dict.pkl','rb'))
question_features = pickle.load(open('data/sample_by_question_questions_features.pkl','rb'))
question_intent_features = pickle.load(open('data/sample_by_question_query_intents.pkl','rb'))
dict_url = pickle.load(open('data/sample_by_question_page_dict.pkl','rb'))
dict_title_embed=pickle.load(open('data/sample_by_question_page_features.pkl','rb'))



association_mat = pickle.load(open('data/all_question_article_ratings.pkl','rb'))

In [13]:
association_mat

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 8),
 (10, 9),
 (11, 10),
 (12, 11),
 (13, 8),
 (14, 12),
 (15, 13),
 (16, 6),
 (17, 14),
 (18, 15),
 (19, 14),
 (20, 6),
 (21, 16),
 (22, 13),
 (23, 13),
 (24, 17),
 (25, 13),
 (26, 17),
 (27, 15),
 (28, 18),
 (29, 14),
 (30, 19),
 (31, 20),
 (32, 21),
 (33, 20),
 (34, 22),
 (35, 8),
 (36, 0),
 (37, 23),
 (38, 8),
 (39, 24),
 (40, 8),
 (41, 25),
 (42, 26),
 (43, 25),
 (44, 25),
 (45, 25),
 (46, 25),
 (47, 25),
 (48, 25),
 (49, 27),
 (50, 25),
 (51, 25),
 (52, 28),
 (53, 25),
 (54, 0),
 (55, 29),
 (56, 30),
 (57, 25),
 (58, 31),
 (59, 31),
 (60, 32),
 (61, 31),
 (62, 25),
 (63, 30),
 (64, 33),
 (65, 26),
 (67, 34),
 (68, 26),
 (69, 35),
 (70, 36),
 (71, 25),
 (72, 26),
 (73, 34),
 (74, 37),
 (75, 34),
 (76, 34),
 (77, 38),
 (78, 29),
 (79, 26),
 (80, 39),
 (81, 40),
 (82, 41),
 (83, 42),
 (84, 42),
 (85, 42),
 (86, 43),
 (87, 42),
 (88, 44),
 (89, 45),
 (90, 46),
 (91, 42),
 (92, 47),
 (93, 42),
 (94, 

In [46]:
import numpy as np
#intent_features = np.empty(shape=(2,), dtype=object)
intent_features[1]=question_intent_features[1] 


[993,
 607,
 709,
 455,
 421,
 906,
 428,
 451,
 269,
 946,
 12,
 837,
 627,
 457,
 846,
 521,
 30,
 67,
 940,
 411,
 315,
 932,
 323,
 195,
 775,
 945,
 173,
 628,
 816,
 192,
 65,
 866,
 586,
 131,
 714,
 437,
 230,
 73,
 157,
 412,
 146,
 682,
 382,
 255,
 246,
 785,
 556,
 404,
 815,
 888,
 512,
 519,
 505,
 434,
 343,
 482,
 136,
 92,
 376,
 122,
 591,
 485,
 284,
 915,
 341,
 687,
 518,
 697,
 874,
 598,
 300,
 197,
 456,
 648,
 529,
 492,
 5,
 72,
 596,
 462,
 886,
 328,
 34,
 344,
 130,
 182,
 544,
 773,
 29,
 674,
 127,
 622,
 893,
 696,
 351,
 372,
 580,
 897,
 86,
 66,
 26,
 568,
 14,
 797,
 561,
 661,
 58,
 752,
 504,
 59,
 589,
 822,
 319,
 632,
 481,
 727,
 202,
 792,
 179,
 239,
 494,
 285,
 420,
 704,
 651,
 280,
 891,
 836,
 890,
 533,
 823,
 368,
 878,
 639,
 348,
 359,
 144,
 203,
 895,
 487,
 210,
 330,
 317,
 110,
 865,
 646,
 81,
 99,
 902,
 951,
 522,
 672,
 909,
 447,
 582,
 352,
 531,
 538,
 142,
 594,
 503,
 75,
 408,
 713,
 48,
 267,
 626,
 576,
 96,
 287,
 5